In [1]:
import numpy as np 

x = np.array([1, 2, 4], dtype = np.float32)
x.itemsize # 4 bytes per float number

4

In [2]:
np.sin(x) # faster than math.sin, no explicit for loop

array([ 0.841471 ,  0.9092974, -0.7568025], dtype=float32)

### NP is limited to 32 dimensions unless we build it for more.

In [13]:
x = np.array([[1, 2, 3], [4, 5, 6]])
print(f"Shape: {x.shape} First Row: {x[0]} First Col: {x[:, 0]}")
print(f"All Rows but stride = 2: \n{x[:, ::2]}")
print(f"Reverse Rows \n{x[::-1]}")
print(f"Reverse Cols \n{x[:, ::-1]}")

Shape: (2, 3) First Row: [1 2 3] First Col: [1 4]
All Rows but stride = 2: 
[[1 3]
 [4 6]]
Reverse Rows 
[[4 5 6]
 [1 2 3]]
Reverse Cols 
[[3 2 1]
 [6 5 4]]


### Numpy uses pass-by-reference semantics
- Slice operation are **views** into the array without implicitly copying 

#### Cases Creating Copies
1. Using a list or a similar sequence type (but not a tuple) for indexing

In [15]:
arr = np.array([1, 2, 3, 4, 5])
indices = [0, 2, 4]  # non-tuple sequence
new_arr = arr[indices]

2. Use another NumPy array, which can be either of integer indices or boolean values, for indexing.

In [16]:
index_arr = np.array([0, 2, 4])
boolean_idx = np.array([True, False, True, False, True])

arr_by_int_np = arr[index_arr]
print(f"By int nparr:{arr_by_int_np}")
arr_by_bool = arr[boolean_idx]
print(f"By boolean nparr:{arr_by_bool}")

By int nparr:[1 3 5]
By boolean nparr:[1 3 5]


3. If you use a tuple for indexing, and **this tuple contains either a sequence (like a list) or a NumPy array**, it also results in a copy.

In [20]:
arr_2d = np.array([[1, 2], [3, 4], [5, 6]])
# Selecting the elements at (0, 1), (2, 1), (1, 1)
indices = ([0, 2, 1], [1, 1, 1])  # Tuple with two lists
new_arr = arr_2d[indices]
print(new_arr)

[2 6 4]


#### We construct y by slicing (which is a view), then any change will affect y
#### ```x.copy()``` explicitly force a copy 
#### ```y.flags``` sort this out

In [26]:
x = np.ones((3, 3))
y = x[:2, :2]
print(f"original y : \n{y}\nThen we make a change to x")
x[0, 0] = 999
y

original y : 
[[1. 1.]
 [1. 1.]]
Then we make a change to x


array([[999.,   1.],
       [  1.,   1.]])

In [27]:
y = x.copy()
y.flags

  C_CONTIGUOUS : True
  F_CONTIGUOUS : False
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False

### Overlapping Blocks (using view. NOT a copy)

Bear in mind that ```as_strided``` **DOESN't** check that you stay within memory block bounds. 
So, if the size of the target matrix is not filled by the available data, the **remaining elements will come from whatever bytes are at that memory location**.

In [28]:
from numpy.lib.stride_tricks import as_strided
x = np.arange(16, dtype = np.int64) # one int is 8-byte
print(f"x : {x}")

# (7, 4) stands for Shape
# (16, 8) stands for strides which means the resulting arr steps 8 bytes in the cols, and steps 16 bytes in the rows
y = as_strided(x, (7, 4), (16, 8))
y

x : [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]


array([[ 0,  1,  2,  3],
       [ 2,  3,  4,  5],
       [ 4,  5,  6,  7],
       [ 6,  7,  8,  9],
       [ 8,  9, 10, 11],
       [10, 11, 12, 13],
       [12, 13, 14, 15]])


#### Multiplication Operation ```A@x``` or ```A.dot(x)```

In [32]:
A = np.array([[1, 2, 3, 4], [3, 4, 5, 6], [6, 7, 8, 9]])
x = np.array([1, 0, 0, 0])
A@x
A.dot(x)

array([1, 3, 6])

### Boardcasting

#### ```None``` tells Numpy to make copies of y along this dimension to create a conformable calculation.

In [39]:
x = np.array([0, 1])
y = np.array([0, 1, 2]).reshape([-1, 1])
print(f"x + y: \n{x + y}")

x = np.array([0, 1])
y = np.array([0, 1, 2])
print("Add one dimension to y and add one dim to x")
x + y[:, None], x[:, None] + y

x + y: 
[[0 1]
 [1 2]
 [2 3]]


(array([[0, 1],
        [1, 2],
        [2, 3]]),
 array([[0, 1, 2],
        [1, 2, 3]]))

In [47]:
z = np.array([0, 1, 2, 3])
x + y[:, None] + z[:, None, None]

array([[[0, 1],
        [1, 2],
        [2, 3]],

       [[1, 2],
        [2, 3],
        [3, 4]],

       [[2, 3],
        [3, 4],
        [4, 5]],

       [[3, 4],
        [4, 5],
        [5, 6]]])